## Tables on a High Level
### Facts:
-- Transactions 69 MB

-- Average Costs 2 MB

### Dim:
-- rtloc 1 KB

-- prod 27 KB

-- pricestate 113 B

-- possite 1 KB

-- invstatus 3 KB

-- invloc 1 KB

-- hldy 297 B

-- clnd 34 KB

## Checking DBFS

In [0]:
dbutils.fs.ls('/')

Out[5]: [FileInfo(path='dbfs:/FileStore/', name='FileStore/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/databricks-datasets/', name='databricks-datasets/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/databricks-results/', name='databricks-results/', size=0, modificationTime=0)]

## Uploaded files to DBFS

In [0]:
dbutils.fs.ls('dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com')

Out[8]: [FileInfo(path='dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/fact_averagecosts_dlm.gz', name='fact_averagecosts_dlm.gz', size=2219727, modificationTime=1741431178000),
 FileInfo(path='dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/fact_transactions_dlm.gz', name='fact_transactions_dlm.gz', size=72346794, modificationTime=1741431202000),
 FileInfo(path='dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/hier_clnd_dlm.gz', name='hier_clnd_dlm.gz', size=35281, modificationTime=1741431179000),
 FileInfo(path='dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/hier_hldy_dlm.gz', name='hier_hldy_dlm.gz', size=297, modificationTime=1741431180000),
 FileInfo(path='dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/hier_invloc_dlm.gz', name='hier_invloc_dlm.gz', size=1047, modificationTime=1741431181000),
 FileInfo(path='dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/hier_invstatus_dlm.gz', name='hier_invstatus_dlm.gz', size=2929, modification

In [0]:
from pyspark.sql.functions import *
options = {
    "delimiter": "|", 
    "inferSchema": "true",
    "header": "true"
}

## Reading Fact Tables and Inferring Schema

In [0]:
df_average_costs= spark.read.format("csv").options(**options).load("dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/fact_averagecosts_dlm.gz")
print(df_average_costs.printSchema())
display(df_average_costs.limit(2))

root
 |-- fscldt_id: integer (nullable = true)
 |-- sku_id: string (nullable = true)
 |-- average_unit_standardcost: double (nullable = true)
 |-- average_unit_landedcost: double (nullable = true)

None


fscldt_id,sku_id,average_unit_standardcost,average_unit_landedcost
20160131,0174410000,10.06,10.06
20160201,0174410000,10.06,10.06


In [0]:
# Null, Uniqueness on Pkey
null_count=df_average_costs.filter(col("fscldt_id").isNull()).count()
print(null_count)
print(df_average_costs.count())
print(df_average_costs.select(countDistinct("fscldt_id","sku_id")).collect()[0][0])

0
740805
740805


In [0]:
df_transcations= spark.read.format("csv").options(**options).load("dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/fact_transactions_dlm.gz")
print(df_transcations.printSchema())
display(df_transcations.limit(2))

root
 |-- order_id: long (nullable = true)
 |-- line_id: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- dt: timestamp (nullable = true)
 |-- pos_site_id: string (nullable = true)
 |-- sku_id: string (nullable = true)
 |-- fscldt_id: integer (nullable = true)
 |-- price_substate_id: string (nullable = true)
 |-- sales_units: integer (nullable = true)
 |-- sales_dollars: double (nullable = true)
 |-- discount_dollars: double (nullable = true)
 |-- original_order_id: long (nullable = true)
 |-- original_line_id: integer (nullable = true)

None


order_id,line_id,type,dt,pos_site_id,sku_id,fscldt_id,price_substate_id,sales_units,sales_dollars,discount_dollars,original_order_id,original_line_id
164087401,2,Sale,2016-01-31T06:17:01.000+0000,CATMAIN,2668940801,20160131,FP,1,58.95,0.0,null,null
164087409,4,Sale,2016-01-31T06:17:25.000+0000,CATMAIN,2920920601,20160131,FP,1,49.95,0.0,null,null


In [0]:
null_count=df_transcations.filter(col("order_id").isNull()).count()
print(null_count)
print(df_transcations.count())
print(df_transcations.select(countDistinct("order_id","line_id","type","dt","pos_site_id","sku_id","fscldt_id","price_substate_id")).collect()[0][0])

0
4503108
4502440


## Reading Dimensions and Inferring Schema

In [0]:
df_clnd= spark.read.format("csv").options(**options).load("dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/hier_clnd_dlm.gz")
print(df_clnd.printSchema())
display(df_clnd.limit(2))

root
 |-- fscldt_id: integer (nullable = true)
 |-- fscldt_label: string (nullable = true)
 |-- fsclwk_id: integer (nullable = true)
 |-- fsclwk_label: string (nullable = true)
 |-- fsclmth_id: integer (nullable = true)
 |-- fsclmth_label: string (nullable = true)
 |-- fsclqrtr_id: integer (nullable = true)
 |-- fsclqrtr_label: string (nullable = true)
 |-- fsclyr_id: integer (nullable = true)
 |-- fsclyr_label: integer (nullable = true)
 |-- ssn_id: string (nullable = true)
 |-- ssn_label: string (nullable = true)
 |-- ly_fscldt_id: integer (nullable = true)
 |-- lly_fscldt_id: integer (nullable = true)
 |-- fscldow: integer (nullable = true)
 |-- fscldom: integer (nullable = true)
 |-- fscldoq: integer (nullable = true)
 |-- fscldoy: integer (nullable = true)
 |-- fsclwoy: integer (nullable = true)
 |-- fsclmoy: integer (nullable = true)
 |-- fsclqoy: integer (nullable = true)
 |-- date: date (nullable = true)

None


fscldt_id,fscldt_label,fsclwk_id,fsclwk_label,fsclmth_id,fsclmth_label,fsclqrtr_id,fsclqrtr_label,fsclyr_id,fsclyr_label,ssn_id,ssn_label,ly_fscldt_id,lly_fscldt_id,fscldow,fscldom,fscldoq,fscldoy,fsclwoy,fsclmoy,fsclqoy,date
20180204,"Feb 4, 2018",201801,"WK 01, 2018",201801,"Feb, 2018",20181,"Q1, 2018",2018,2018,SPRG2018,Spring 2018,20170205,20160207,1,1,1,1,1,1,1,2018-02-04
20180205,"Feb 5, 2018",201801,"WK 01, 2018",201801,"Feb, 2018",20181,"Q1, 2018",2018,2018,SPRG2018,Spring 2018,20170206,20160208,2,2,2,2,1,1,1,2018-02-05


In [0]:
null_count=df_clnd.filter(col("fscldt_id").isNull()).count()
print(null_count)
print(df_clnd.count())
print(df_clnd.select(countDistinct("fscldt_id")).collect()[0][0])

0
1820
1820


In [0]:
df_hldy= spark.read.format("csv").options(**options).load("dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/hier_hldy_dlm.gz")
print(df_hldy.printSchema())
display(df_hldy.limit(2))

root
 |-- hldy_id: string (nullable = true)
 |-- hldy_label: string (nullable = true)

None


hldy_id,hldy_label
Valentines_Day,Valentine's Day
Presidents_Day,Presidents Day


In [0]:
null_count=df_hldy.filter(col("hldy_id").isNull()).count()
print(null_count)
print(df_hldy.count())
print(df_hldy.select(countDistinct("hldy_id")).collect()[0][0])

0
19
19


In [0]:
df_invloc = spark.read.format("csv").options(**options).load("dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/hier_invloc_dlm.gz")
print(df_invloc.printSchema())
display(df_invloc.limit(2))

root
 |-- loc: integer (nullable = true)
 |-- loc_label: string (nullable = true)
 |-- loctype: string (nullable = true)
 |-- loctype_label: string (nullable = true)

None


loc,loc_label,loctype,loctype_label
103,Catalog/Internet,DC,DC
119,Retail,DC,DC


In [0]:
null_count=df_invloc.filter(col("loc").isNull()).count()
print(null_count)
print(df_invloc.count())
print(df_invloc.select(countDistinct("loc")).collect()[0][0])

0
85
85


In [0]:
df_invstatus = spark.read.format("csv").options(**options).load("dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/hier_invstatus_dlm.gz")
print(df_invstatus.printSchema())
display(df_invstatus.limit(2))

root
 |-- code_id: string (nullable = true)
 |-- code_label: string (nullable = true)
 |-- bckt_id: string (nullable = true)
 |-- bckt_label: string (nullable = true)
 |-- ownrshp_id: string (nullable = true)
 |-- ownrshp_label: string (nullable = true)

None


code_id,code_label,bckt_id,bckt_label,ownrshp_id,ownrshp_label
INV-880,In-Transit,IBIT,In-transit,IB,Inbound
INV-875,ASN,IBOO,On Order,IB,Inbound


In [0]:
null_count=df_invstatus.filter(col("code_id").isNull()).count()
print(null_count)
print(df_invstatus.count())
print(df_invstatus.select(countDistinct("code_id")).collect()[0][0])

0
245
245


In [0]:
df_possite = spark.read.format("csv").options(**options).load("dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/hier_possite_dlm.gz")
print(df_possite.printSchema())
display(df_possite.limit(2))

root
 |-- site_id: string (nullable = true)
 |-- site_label: string (nullable = true)
 |-- subchnl_id: string (nullable = true)
 |-- subchnl_label: string (nullable = true)
 |-- chnl_id: string (nullable = true)
 |-- chnl_label: string (nullable = true)

None


site_id,site_label,subchnl_id,subchnl_label,chnl_id,chnl_label
CATMAIN,Catalog Main,CATMAIN,Catalog Main,CAT,Catalog
CATPROSP,Catalog Prospect,CATPROSP,Catalog Prospect,CAT,Catalog


In [0]:
null_count=df_possite.filter(col("site_id").isNull()).count()
print(null_count)
print(df_possite.count())
print(df_possite.select(countDistinct("site_id")).collect()[0][0])

0
90
90


In [0]:
df_pricestate = spark.read.format("csv").options(**options).load("dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/hier_pricestate_dlm.gz")
print(df_pricestate.printSchema())
display(df_pricestate.limit(2))

root
 |-- substate_id: string (nullable = true)
 |-- substate_label: string (nullable = true)
 |-- state_id: string (nullable = true)
 |-- state_label: string (nullable = true)

None


substate_id,substate_label,state_id,state_label
FP,Full Price,FP,Full Price
MD1,Markdown 1,MD,Markdown


In [0]:
null_count=df_pricestate.filter(col("substate_id").isNull()).count()
print(null_count)
print(df_pricestate.count())
print(df_pricestate.select(countDistinct("substate_id")).collect()[0][0])

0
4
4


In [0]:
df_prod = spark.read.format("csv").options(**options).load("dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/hier_prod_dlm.gz")
print(df_prod.count())
print(df_prod.printSchema())
display(df_prod.limit(2))

1235
root
 |-- sku_id: string (nullable = true)
 |-- sku_label: string (nullable = true)
 |-- stylclr_id: string (nullable = true)
 |-- stylclr_label: string (nullable = true)
 |-- styl_id: string (nullable = true)
 |-- styl_label: string (nullable = true)
 |-- subcat_id: integer (nullable = true)
 |-- subcat_label: string (nullable = true)
 |-- cat_id: integer (nullable = true)
 |-- cat_label: string (nullable = true)
 |-- dept_id: integer (nullable = true)
 |-- dept_label: string (nullable = true)
 |-- issvc: integer (nullable = true)
 |-- isasmbly: integer (nullable = true)
 |-- isnfs: integer (nullable = true)

None


sku_id,sku_label,stylclr_id,stylclr_label,styl_id,styl_label,subcat_id,subcat_label,cat_id,cat_label,dept_id,dept_label,issvc,isasmbly,isnfs
2230940206,COTTON UNDERWIRE CAMI WHITE 36C,2230940,COTTON UNDERWIRE CAMI WHITE,22309,COTTON UNDERWIRE CAMI,405,TOPS,1000,TOPS,2000,APPAREL,0,0,0
2230940208,COTTON UNDERWIRE CAMI WHITE 38C,2230940,COTTON UNDERWIRE CAMI WHITE,22309,COTTON UNDERWIRE CAMI,405,TOPS,1000,TOPS,2000,APPAREL,0,0,0


In [0]:
null_count=df_prod.filter(col("sku_id").isNull()).count()
print(null_count)
print(df_prod.count())
print(df_prod.select(countDistinct("sku_id")).collect()[0][0])

0
1235
1235


In [0]:
df_rtloc = spark.read.format("csv").options(**options).load("dbfs:/FileStore/shared_uploads/ksivareddy1997@gmail.com/hier_rtlloc_dlm.gz")
print(df_rtloc.count())
print(df_rtloc.printSchema())
display(df_rtloc.limit(2))

84
root
 |-- str: integer (nullable = true)
 |-- str_label: string (nullable = true)
 |-- dstr: integer (nullable = true)
 |-- dstr_label: string (nullable = true)
 |-- rgn: integer (nullable = true)
 |-- rgn_label: string (nullable = true)

None


str,str_label,dstr,dstr_label,rgn,rgn_label
115,STL County Library,0,No District Name,0,No Region Name
126,Burr Ridge,0,No District Name,0,No Region Name


In [0]:
null_count=df_rtloc.filter(col("str").isNull()).count()
print(null_count)
print(df_rtloc.count())
print(df_rtloc.select(countDistinct("str")).collect()[0][0])

0
84
84


### Foreign Key Constraints Between Fact and Dimension Tables

In [0]:
df_transaction_possite=df_transcations.alias("t").join(df_possite.alias("ps"), col("t.pos_site_id") == col("ps.site_id"), "left_anti")
display(df_transaction_possite.limit(2))


order_id,line_id,type,dt,pos_site_id,sku_id,fscldt_id,price_substate_id,sales_units,sales_dollars,discount_dollars,original_order_id,original_line_id


In [0]:
df_transaction_prod=df_transcations.alias("t").join(df_prod.alias("ps"), col("t.sku_id") == col("ps.sku_id"), "left_anti")
display(df_transaction_possite.limit(2))

order_id,line_id,type,dt,pos_site_id,sku_id,fscldt_id,price_substate_id,sales_units,sales_dollars,discount_dollars,original_order_id,original_line_id


### Dump

In [0]:
# df_possite.select('site_id').distinct().show()
# df_average_costs.groupBy("fscldt_id") \
#     .count() \
#     .filter(col("count") > 1) \
#     .orderBy(col("fscldt_id")) \
#     .show()
# df_average_costs.select(col("fscldt_id")) \
#     .where(col("fscldt_id") == "20160204") \
#     .show()
# df_transaction_possite.where(col('pos_site_id')=='2668940801').show()
# df_possite.where(col('site_id')=='2668940801').show()
# df_transaction_possite.select(col('sku_id')).show(2)
# df_possite.select(col('site_id')).show(2)